# **Exposure Time Calculator**

In [1]:
# Import modules
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from glob import glob
import pandas as pd
from scipy.interpolate import interp1d

In [10]:
## Filters
file_path = 'files/'
filt_trans_df = pd.read_table(os.path.join(file_path, 'TransFiltersSDSS'))

## Detectors
QHY411_QE_df  = pd.read_csv(os.path.join(file_path, 'QE_QHY411M.csv'))
iKon936_QE_df = pd.read_csv(os.path.join(file_path, 'QE_iKon936.csv'))
Camelot_QE_df = pd.read_csv(os.path.join(file_path, 'QE_Camelot.csv'))
Camelot_QE_df = Camelot_QE_df.append(pd.DataFrame({'Wavelength (nm)': np.linspace(1000, 1110, 11), 'QE (%)': np.zeros(11)}), ignore_index=True)

## Extinction coefficients for Teide Observatory
ext_dic = {'SDSSr' : 0.12,
           'SDSSi' : 0.09,
           'SDSSg' : 0.15,}


/var/folders/6k/7lpgck413j1__xdr1fy86f9h0000gn/T/ipykernel_52381/2242107912.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Camelot_QE_df = Camelot_QE_df.append(pd.DataFrame({'Wavelength (nm)': np.linspace(1000, 1110, 11), 'QE (%)': np.zeros(11)}), ignore_index=True)


### **System response function**

The capture of photons by the optical system depends on the collecting power of the telescope and the succession of elements in the optical path, which produce a wavelength-dependent transmittance drop.

Regarding the telescope, the collecting power will be considered to be flat over the whole spectrum covered by the instruments, a parameter $\epsilon$ is introduced that can be understood as the performance of the mirrors and lenses. **This parameter should be modeled once the telescope is installed**. For the time being, it will be set, according to the specifications, at 96%.


$$
S = \epsilon \frac{\pi}{4}D^2(1-O^2) \frac{1}{h}\int_0^\infty S^{filt}(\lambda)QE(\lambda) \lambda^{-1} d\lambda
$$

where $D$ is the diameter of the telescope, $O$ the central obstruction percentage, $h = 6.626\times 10^{-34}$ $J/s$, $S^{filt}$ the filter transmittance and $QE$ the detector quantum efficiency.

In [11]:
def S (D, filt, filt_trans, QE, O=0.41, eps=0.96):
  # Input:
  #   D           :  Telescope diameter (m)
  #   filt_trans  :  Filter transmitance (%)
  #   QE          :  Quantum efficiency (%)
  #   O           :  Telescope obscuration (%)
  #   eps         :  Telescope efficiency (%)
  # Return:
  #   S_filt      :  System integral (m2 s/J)

  # Calculate the system integral
  x = filt_trans['Wavelength (nm)'].to_numpy()
  y_filt = filt_trans[filt].to_numpy()*1e-2
  f = interp1d(QE['Wavelength (nm)'], QE['QE (%)']*1e-2)   # interpolate QE
  y_QE = f(x)
  int_lambda = np.trapz(y_filt * y_QE * 1./x,x)            # numeric integration

  S_filt = eps * np.pi/4 * D**2 * (1-O**2) / (6.626e-34) * int_lambda
  return S_filt

### **Object photometry**

A source with flat spectral energy distribution, $F_\nu(\lambda) = F_o$, is assumed for standardized SNR calculation. Given its corresponding AB magnitude:
$$
m_{AB} = −2.5 log_{10}(F_o/3631 Jy)
$$

the density flux received per second on the top of the atmosphere is:

$$
F_o = 3631 Jy \cdot 10^{-0.4 m_{AB}}
$$

In [12]:
def F (m):
  # Input:
  #   m      :  AB magnitude
  # Return:
  #   F_0    :  Density flux (J/s/Hz/m2)

  return 3631e-26 * 10**(-0.4 * m)

### **Atmospheric extinction**

The reduction of the photon flux produced by the atmosphere is known as extinction. Given the zenital distance $ZD$ of an object, the airmass $\chi$ is defined as:
$$
\chi = sec(h)
$$

And the extinction term:
$$
E = 10^{-0.4 \chi \kappa_b}
$$

where $\kappa_b$ is the extinction coefficient in the filter $b$, which is characteristic of the observation site and may undergo small variations during the night. The absence of these variations is known as "photometric night".

In [13]:
def E(ZD, k):
  # Input:
  #   ZD      :  Zenithal distance (deg)
  #   k       :  Extinction coefficient (mag/airmass)
  # Return:
  #   E       :  Extinction term

  X = 1/np.cos(ZD * np.pi/180)
  return 10 ** (-0.4 * X * k)

### **Seeing**

The aperture area in photometry depend on the FWHM of the object. In general, this will vary with wavelength and airmass:
$$
S = S_{0} \cdot \chi^{0.6} \cdot \left(\frac{\lambda}{\lambda_0}\right)^{-0.2}
$$

where $S_0$ is the seeing measured at the zenith at wavelength $\lambda_0$.

In [14]:
def Seeing(S0, ZD, l, l0=900):
  # Input:
  #   S0      :  Seeing (arcsec) at the zenith at wavelength l0 (nm)
  #   ZD      :  Zenithal distance (deg)
  #   l       :  Desired wavelength (nm)
  # Return:
  #   Sc      :  Corrected seeing (arcsec)

  X = 1/np.cos(ZD * np.pi/180)
  return S0 * X**(0.6) * (l/l0)**(-0.2)

### **Signal**

The total electrons per second measured by the detector at the telescope focal plane is, taking into account the above contributions:

$$
C_b = F_o \cdot E \cdot S = \epsilon \frac{\pi}{4}D^2(1-O^2) \frac{1}{h} 10^{-0.4 (m_{AB} +\chi \kappa_b)}\int_0^\infty S^{filt}(\lambda)QE(\lambda) \lambda^{-1} d\lambda
$$

Equivalently, having the sky brightness $m_{sky}$ in units of magnitude per arcsec$^2$, the number of electrons per second in each pixel is:

$$
B_b = \epsilon \frac{\pi}{4}D^2(1-O^2) \frac{1}{h} 10^{-0.4 m_{sky}} scale^2 \int_0^\infty S^{filt}(\lambda)QE(\lambda) \lambda^{-1} d\lambda
$$

In [15]:
def Signal(m_AB, m_sky, ZD, k, D, filt, filt_trans, QE, O=0.41, eps=0.96, scale=0.2):
  # Input:
  #   m_AB        :  Target AB magnitude
  #   m_sky       :  Sky background AB magnitude
  #   ZD          :  Object zenital distance (deg)
  #   k           :  Extinction coefficient in the filter band (mag/airmass)
  #   D           :  Telescope diameter (m)
  #   filt_trans  :  Filter transmitance (%)
  #   QE          :  Quantum efficiency (%)
  #   O           :  Telescope obscuration (%)
  #   eps         :  Telescope efficiency (%)
  #   scale       :  Pixel scale (arcsec/px)
  # Return:
  #   C_b         :  Object detected e-/s
  #   B_b         :  Sky detected e-/s

  system = S (D, filt, filt_trans, QE, O, eps)
  flux_obj = F (m_AB)
  flux_sky = F (m_sky)
  ext = E(ZD, k)
  C_b = flux_obj * system * ext
  B_b = flux_sky * system * scale**2
  return C_b, B_b

### **Signal-to-Noise Ratio**

The main noise components include photonic noise due to the object, the sky and the dark current $DC$, as well as Gaussian readout noise $R$. The signal-to-noise ratio is:

$$
SNR = \frac{C_b t_{exp}}{\sqrt{C_b t_{exp} + n_{p}(B_b t_{exp} + DC t_{exp} + R^2)}}
$$

where $t_{exp}$ is the exposure time and $n_p = 2.266(FWHM/scale)^2$ the number of pixels considered on the photometry aperture.

The exposure time can then be found by using the expression:

$$
t_{exp} = \frac{SNR^2[C_b+n_{p}(B_b+DC)] + \sqrt{SNR^4[C_b+n_{p}(B_b+DC)]^2 + 4C_b^2SNR^2n_{p}R^2}}{2C_b^2}
$$

In [16]:
def SNR_tg(C_b, B_b, DC, R, texp, FWHM, scale):
  # Input:
  #   C_b         :  Object detected e-/s
  #   B_b         :  Sky detected e-/s
  #   DC          :  Detector dark current (e-/s)
  #   R           :  Detector readout noise (e-)
  #   texp        :  Exposure time (s)
  #   FWHM        :  FWHM of the star psf (arcsec)
  #   scale       :  Pixel scale (arcsec/px)
  # Return:
  #   SNR         :  Signal-to-Noise ratio of the targe photometry

  n_p = 2.266 * (FWHM/scale)**2
  return C_b * texp / np.sqrt(C_b * texp + n_p * (B_b * texp + DC * texp + R**2))

def texp (C_b, B_b, DC, R, SNR, FWHM, scale):
  # Input:
  #   C_b         :  Object detected e-/s
  #   B_b         :  Sky detected e-/s
  #   DC          :  Detector dark current (e-/s)
  #   R           :  Detector readout noise (e-)
  #   SNR         :  Signal-to-Noise ratio of the targe photometry
  #   FWHM        :  FWHM of the star psf (arcsec)
  #   scale       :  Pixel scale (arcsec/px)
  # Return:
  #   texp        :  Exposure time (s)

  n_p = 2.266 * (FWHM/scale)**2
  t = (SNR**2 * (C_b + n_p * (B_b + DC)) +
       np.sqrt(SNR**4 * (C_b + n_p * (B_b + DC))**2 +
               4 * C_b**2 * SNR**2 * n_p * R**2)
       ) / (2 * C_b**2)
  return t

def plate_scale_mm(focal):
    # focal length in mm
    return 206265 / focal # arcsec/mm

def plate_scale_px(microns, focal):
    # pixel size in microns
    return plate_scale_mm(focal) * microns/1000  # arcsec/px

### **Co-adding images**

If several exposures are coadded in a single **averaged** image, the SNR improves with the square root of the number of sub-exposures $N$:

$$
SNR_{coadded} = SNR \cdot \sqrt{N}
$$

In [17]:
def coadd(SNR, N, method='avg'):
  # Input:
  #   SNR           :  Signal-to-Noise Ratio in a single image
  #   N             :  Number of coadded exposures
  #   method        :  Co-adding method
  # Return:
  #   SNR_coadded   :  Equivalent Signal-to-Noise Ratio in the co-added image
  if method == 'avg': SNR_coadded = SNR * np.sqrt(N)

  return SNR_coadded

def N_coadded(SNR_max, SNR):
  N = (SNR / SNR_max)**2
  return N

In addition to this, observing without autoguiding and a high dark current sets a limit on the maximum exposure time. We will therefore distinguish integration time $t_{integ}$ as the equivalent time of the image formed by the co-adding of $N$ exposures with $t_{exp}$:
$$
t_{integ} = N \cdot t_{exp}
$$

In [21]:
def ETC(m_AB, m_sky, SNR, filt, ZD, k, S0, texp_max, det='QHY411', tel='AZ800'):
  # Input:
  #   m_AB      :  Target AB magnitude
  #   m_sky     :  List of sky background AB magnitudes
  #   SNR       :  Desired Signal-to-Noise ratio of the target
  #   filt      :  Filter name (SDSSr, SDSSg, SDSSi)
  #   ZD        :  List of object zenital distance (deg)
  #   k         :  List of extinction coefficient in the filter band (mag/airmass)
  #   S0        :  List of Seeing at 900nm at zenith (arcsec)
  #   texp_max  :  Maximum exposure time (s)
  #   det       :  Detector name (QHY411, Ikon936, Camelot)
  #   tel       :  Telescope name (AZ800, AZ2000, IAC80)
  # Return:
  #   t_exp     :  Exposure time (s)
  #   N_exp     :  Number of exposures

  # Telescope characteristics
  if tel == 'AZ800':
    D = 0.8
    fD = 6.85
    O = 0.41
    eps = 0.5
  elif tel == 'AZ2000':
    D = 2
    fD = 6.85
    O = 0.41
    eps = 0.5
  elif tel == 'IAC80':
    D = 0.82
    fD = 11.3
    O = 0.55
    eps = 0.45
  else:
    print('Telescope is not available')
    return None

  # Detector characteristics
  if det == 'QHY411':
    QE = QHY411_QE_df[['Wavelength (nm)','QE (%)']].sort_values('Wavelength (nm)')
    px_sz = 3.76
    fD *= 0.64
    R = 3.74
    DC = 0.0032
  elif det == 'iKon936':
    QE = iKon936_QE_df[['Wavelength (nm)','QE (%)']].sort_values('Wavelength (nm)')
    px_sz = 13.5
    R = 7
    DC = 0.0003
  elif det == 'Camelot':
    QE = Camelot_QE_df[['Wavelength (nm)','QE (%)']].sort_values('Wavelength (nm)')
    px_sz = 15
    R = 12.11
    DC = 0.0003
  else:
    print('Detector is not available')
    return None

  # Read filter transmitance
  try: filt_trans = filt_trans_df[['Wavelength (nm)',filt]].sort_values('Wavelength (nm)')
  except:
    print('Filter is not available')
    return None

  # Calculate telescope scale
  scale = plate_scale_px(px_sz, D * 1e3 * fD)

  # Correct FWHM from zenithal distance and wavelength
  FWHM = Seeing(S0, ZD, np.mean(filt_trans.loc[filt_trans[filt] > 0.1, 'Wavelength (nm)']))

  # Obtain signals
  C_b, B_b = Signal(m_AB, m_sky, ZD, k, D, filt, filt_trans, QE, O, eps, scale)
  SNR_max = SNR_tg(C_b, B_b, DC, R, texp_max, FWHM, scale)
  N = np.ceil(N_coadded(SNR_max, SNR))
  if N == 1:
    t_exp = texp (C_b, B_b, DC, R, SNR, FWHM, scale)
  else:
    t_exp = texp (C_b, B_b, DC, R, SNR / np.sqrt(N), FWHM, scale)

  return t_exp, N

### Usage

In [22]:
m_AB = 20
m_sky = 20
SNR = 62
filt = 'SDSSr'
ZD = 20
k = ext_dic[filt]
texp_max = 300
S0 = 2

exptime, N = ETC(m_AB, m_sky, SNR, filt, ZD, k, S0, texp_max, det='QHY411', tel='AZ800')

290.6350522798231 18.0
